# Quick start

Let's set up a simple emulator for scattering of 14.1 MeV protons on $^{27}$Al, using the [Koning-Delaroche](https://www.sciencedirect.com/science/article/pii/S0375947402013210?casa_token=qS1v6U4xDQEAAAAA:NIi9D5LpP3f05AMwRnvbQ6or8hSvXoEIgKBV56KA4l9aObCOVDAndmuCeIH77iuzoXMOOlAMyw) optical potential for the proton-nucleus interaction.

In [1]:
import rose
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt

In [2]:
# set up kinematics
from rose.koning_delaroche import KDGlobal, Projectile

# for 27-Al
A = 27
Z = 13

# lab bombarding energy
Elab = 14.1  # MeV

# get kinematics and default KD params
omp = rose.koning_delaroche.KDGlobal(Projectile.proton)
(mu, Ecom, k, eta, R_C), parameters = omp.get_params(A, Z, Elab)

In [3]:
# take train and test parameter samples from a box bounded
# 50% above and below the default values
scale = 0.5

bounds = np.array(
    [
        parameters - np.fabs(scale * parameters),
        parameters + np.fabs(scale * parameters),
    ]
).T

In [4]:
from rose.training import sample_params_LHC

# sample points for a train/test split
# sample from +/-50% around the default params using Latin hypercube
n_test = 20
n_train = 80
train = sample_params_LHC(n_train, parameters, scale, seed=13)
test = sample_params_LHC(n_test, parameters, scale, seed=721)

Instantiate the set of interactions for each partial wave. We use the type `InteractionEIMSpace` because we will use the Empirical Interpolation Method (EIM) to handle potentials that are non-affine (like the Woods-Saxon forms in Koning-Delaroche). We will not discuss this here, for more detail, see the following tutorials.

In [ ]:
interactions = rose.InteractionEIMSpace(
    rose.koning_delaroche.KD_simple,
    len(parameters),
    mu,
    Ecom,
    is_complex=True,
    spin_orbit_potential=rose.koning_delaroche.KD_simple_so,
    training_info=bounds,
    Z_1=1,
    Z_2=13,
    R_C=R_C,
)

Now that we have our `InteractionSpace` set up, we can construct a `ScatteringAmplitudeEmulator`, which trains and runs the emulator for us. We will pass in our set of training points and our angular mesh, and choose a number of basis elements that is the dimension of our reduced basis. In general, this number should be much smaller than the number of training points; increasing it will lead to a more accurate, albeit slower, emulator. In the following tutorial, we will optimize this, amongst other, parameters, to achieve a given accuracy and speedup goal. For now, however, we will choose an ad-hoc basis size of 10, which, it will turn out, will give us a very accurate and fast emulator.

In [ ]:
# set up and train emulator using sensible defaults
emulator = rose.ScatteringAmplitudeEmulator.from_train(
    interactions,
    train,
    n_basis=10,  # we will just choose 10 out of a hat for now, more on this later
    angles=np.linspace(np.pi / 181, np.pi, 180),
)

It's as simple as that, now we have a trained emulator we can use to calculate cross sectons! Let's compare observables between the emulator and the high-fidelity solver using our test set of sampled parameters. We will do this by calling the functions `emulate_xs` and `exact_xs` on our emulator, passing in a set of parameters from our test set.

In [ ]:
%%time
# use %%timeit to get a more percise timing  - but may take a while to run!

# run high-fidelity solver
HIFI_xs = []
for sample in tqdm(test):
    HIFI_xs.append(emulator.exact_xs(sample))

In [ ]:
%%time
# use %%timeit to get a more percise timing  - this one will be fast either way!

# run emulator
emu_xs = []
for sample in tqdm(test):
    emu_xs.append(emulator.emulate_xs(sample))

Now let's plot the results!

In [ ]:
from matplotlib.lines import Line2D

fig, ax = plt.subplots(figsize=(6, 4), dpi=300)
fig.patch.set_facecolor("white")

for i, sample in enumerate(test):
    x = emulator.angles * 180 / (np.pi)

    # HIFI
    y_exact = HIFI_xs[i].dsdo

    # emulated
    y_emu = emu_xs[i].dsdo

    p = ax.plot(x, y_emu, "--", alpha=0.5)
    ax.plot(x, y_exact, color=p[0].get_color(), alpha=0.5)

legend_styles = [
    Line2D([0], [0], color="k", linestyle="--", alpha=0.5),
    Line2D([0], [0], color="k", alpha=0.5),
]
ax.legend(legend_styles, ["emulated", "high-fidelity"])
ax.set_yscale("log")
plt.xlim([0, 180])
plt.xlabel(r"$\theta$ [$^\circ$]", fontsize=16)
plt.ylabel(r"$\frac{d \sigma}{d\Omega}$ [mb/Sr]", fontsize=16)

plt.ylabel(r"$\sigma / \sigma_{\rm{Rutherford}}$ ", fontsize=16)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), dpi=300)
fig.patch.set_facecolor("white")

for i, sample in enumerate(test):
    x = emulator.angles * 180 / (np.pi)

    # HIFI
    y_exact = HIFI_xs[i].Ay

    # emulated
    y_emu = emu_xs[i].Ay

    p = ax.plot(x, y_emu, "--", alpha=0.5)
    ax.plot(x, y_exact, color=p[0].get_color(), alpha=0.5)

legend_styles = [
    Line2D([0], [0], color="k", linestyle="--", alpha=0.5),
    Line2D([0], [0], color="k", alpha=0.5),
]
ax.legend(legend_styles, ["emulated", "high-fidelity"])
ax.set_ylim([-1, 1])
plt.xlim([0, 180])
plt.xlabel(r"$\theta$ [$^\circ$]", fontsize=16)
plt.ylabel(r"$A_y(\theta)$ ", fontsize=16)
plt.show()

Isn't that nice, the emulated and high-fidelity observables are right on top of eachother! The following series of tutorials will walk through how to choose the the optimal hyper-parameters (like the basis size), and demonstrate linking to `surmise` for fast Bayesian uncertainty quantification!